For motivating examples in DVI
We want to compare the difference between
200-205 epochs with pure training data
200-205 training data with wrong testing data,
We will observe the boundaries and positions of data afterward

In [1]:
# import modules
import numpy as np
import torch
import torchvision.transforms as T
from cifar10_models.resnet import resnet18
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [2]:
torch.manual_seed(1331)

train_transform = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])
test_transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])
trainset = datasets.CIFAR10(root="data", train=True, transform=train_transform, download=True)
trainloader = DataLoader(
    trainset,
    batch_size=200,
    num_workers=2,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)
device = torch.device("cuda:0")

Files already downloaded and verified


In [3]:
criterion = nn.CrossEntropyLoss()

In [6]:
net = resnet18()
path = "E:\\DVI_exp_data\\resnet18_cifar10\\Model\\Epoch_{}\\subject_model.pth".format(180)
net.load_state_dict(torch.load(path))
net.to(device)
net.train()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01, nesterov=True)

In [5]:
for epoch in range(20):  # loop over the dataset multiple times
    loop = tqdm(enumerate(trainloader), total=len(trainloader), leave=True)
    for i, data in loop:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _,predictions = outputs.max(1)
        num_correct = (predictions == labels).sum()
        running_train_acc = float(num_correct) / float(inputs.shape[0])
        # print statistics
        loop.set_description(f" Epoch [{epoch+1} / 20]")
        loop.set_postfix(loss=loss.item(), acc=running_train_acc)
    torch.save(net.state_dict(), "consecutive_epochs\\normal_{}.pth".format(epoch+1))

print('Finished Training')

 Epoch [20 / 20]: 100%|██████████| 250/250 [00:31<00:00,  7.95it/s, acc=0.975, loss=0.115] 


Finished Training


In [6]:
trainset = datasets.CIFAR10('data', train=True, transform=test_transform, download=True)
testset = datasets.CIFAR10('data', train=False, transform=test_transform, download=True)

# net = resnet18()
# net.load_state_dict(torch.load(path))
# net.to(device)
net.eval()

testloader = DataLoader(
    testset,
    batch_size=200,
    num_workers=0,
    shuffle=False,
)
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in tqdm(testloader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d images: %d %%' % (total,
        100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:02<00:00, 19.52it/s]

Accuracy of the network on the 10000 images: 89 %


retrain on test samples

In [7]:
# record index on where model has a wrong prediction

trainset = datasets.CIFAR10('data', train=True, transform=test_transform, download=True)
testset = datasets.CIFAR10('data', train=False, transform=test_transform, download=True)

net = resnet18()
net.load_state_dict(torch.load(path))
net.to(device)
net.eval()

testloader = DataLoader(
    testset,
    batch_size=200,
    num_workers=0,
    shuffle=False,
)
correct = 0
total = 0
l=[]
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in tqdm(enumerate(testloader), total=len(testloader), leave=True):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        l.extend((predicted != labels).tolist())

print('Accuracy of the network on the %d images: %d %%' % (total,
                                                           100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:03<00:00, 13.11it/s]

Accuracy of the network on the 10000 images: 92 %


In [8]:
np.sum(l)

743

In [9]:
import numpy as np
tmp=np.arange(10000)
mask = np.array(l)
idxs = tmp[mask]

In [10]:
from PIL import Image
from torch.utils.data import Dataset
class DataHandler(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        if self.transform is not None:
            x = Image.fromarray(x)
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.X)

In [14]:
data_tr = datasets.CIFAR10('data', train=True,  download=True)
data_te = datasets.CIFAR10('data', train=False, download=True)
X_tr = data_tr.data
Y_tr = torch.from_numpy(np.array(data_tr.targets))
X_te = data_te.data[idxs]
Y_te = torch.from_numpy(np.array(data_te.targets))[idxs]

X = np.concatenate((X_tr, X_te), axis=0)
Y = torch.cat((Y_tr, Y_te), axis=0)
datahandler = DataHandler(X, Y, train_transform)
loader = DataLoader(
    datahandler,
    batch_size=200,
    num_workers=0,
    shuffle=True,
)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
net = resnet18()
path = "E:\\DVI_exp_data\\resnet18_cifar10\\Model\\Epoch_{}\\subject_model.pth".format(180)
net.load_state_dict(torch.load(path))
net.to(device)
net.train()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01, nesterov=True)
for epoch in range(20):  # loop over the dataset multiple times
    loop = tqdm(enumerate(loader), total=len(loader), leave=True)
    for (i, data) in loop:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device=device,dtype=torch.long)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _,predictions = outputs.max(1)
        num_correct = (predictions == labels).sum()
        running_train_acc = float(num_correct) / float(inputs.shape[0])
        # print statistics
        loop.set_description(f" Epoch [{epoch+1} / 20]")
        loop.set_postfix(loss=loss.item(), acc=running_train_acc)
    torch.save(net.state_dict(), "consecutive_epochs\\revised_{}.pth".format(epoch+1))

print('Finished Training')

 Epoch [20 / 20]: 100%|██████████| 254/254 [00:39<00:00,  6.45it/s, acc=0.958, loss=0.161] 


Finished Training


In [16]:
# record index on where model has a wrong prediction

trainset = datasets.CIFAR10('data', train=True, transform=test_transform, download=True)
testset = datasets.CIFAR10('data', train=False, transform=test_transform, download=True)

# net = resnet18()
# net.load_state_dict(torch.load(path))
# net.to(device)
net.eval()

testloader = DataLoader(
    testset,
    batch_size=200,
    num_workers=0,
    shuffle=False,
)
correct = 0
total = 0
l = []
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in tqdm(enumerate(testloader), total=len(testloader), leave=True):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        l.extend((predicted != labels).tolist())

print('Accuracy of the network on the %d images: %d %%' % (total,
                                                           100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:02<00:00, 18.71it/s]

Accuracy of the network on the 10000 images: 93 %


In [17]:
np.sum(l)

671

In [18]:
import json
with open("list_index.json", "w") as f:
    json.dump(l,f)